In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('./data.csv')

In [3]:
X,y = data.drop('target',axis=1),data['target']

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.25)

In [6]:
import torch
import torch.nn as nn

In [7]:
import numpy as np

In [8]:
X_train = torch.from_numpy(np.array(X_train).astype(np.float32))
y_train = torch.from_numpy(np.array(y_train).astype(np.float32))
X_test = torch.from_numpy(np.array(X_test).astype(np.float32))
y_test = torch.from_numpy(np.array(y_test).astype(np.float32))

In [9]:
X_train.shape

torch.Size([227, 13])

In [10]:
X_test.shape

torch.Size([76, 13])

In [11]:
y_train.shape

torch.Size([227])

In [12]:
y_test.shape

torch.Size([76])

In [13]:
import torch.nn.functional as F

In [14]:
class Test_Model(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(13,64)
        self.fc2 = nn.Linear(64,128)
        self.fc3 = nn.Linear(128,256)
        self.fc4 = nn.Linear(256,512)
        self.fc5 = nn.Linear(512,1024)
        self.fc6 = nn.Linear(1024,512)
        self.fc7 = nn.Linear(512,1)
    
    def forward(self,X):
        preds = self.fc1(X)
        preds = F.relu(preds)
        preds = self.fc2(preds)
        preds = F.relu(preds)
        preds = self.fc3(preds)
        preds = F.relu(preds)
        preds = self.fc4(preds)
        preds = F.relu(preds)
        preds = self.fc5(preds)
        preds = F.relu(preds)
        preds = self.fc6(preds)
        preds = F.relu(preds)
        preds = self.fc7(preds)
        return F.sigmoid(preds)

In [15]:
device = torch.device('cuda')

In [16]:
X_train = X_train.to(device)
y_train = y_train.to(device)
X_test = X_test.to(device)
y_test = y_test.to(device)

In [17]:
PROJECT_NAME = 'Heart-Disease-UCI'

In [18]:
def get_loss(criterion,X,y,model):
    model.eval()
    with torch.no_grad():
        preds = model(X.float().to(device))
        preds = preds.to(device)
        y = y.to(device)
        loss = criterion(preds,y)
    model.train()
    return loss.item()
def get_accuracy(X,y,model):
    model.eval()
    with torch.no_grad():
        correct = 0
        total = 0
        for i in range(len(X)):
            pred = model(X[i].float().to(device))
            pred.to(device)
            if round(int(pred[0])) == round(int(y[i])):
                correct += 1
            total += 1
        if correct == 0:
            correct += 1
    model.train()
    return round(correct/total,3)

In [19]:
import wandb

In [20]:
from tqdm import tqdm

In [21]:
EPOCHS = 100

In [22]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [23]:
preds

tensor([3.7035e-02, 1.4371e-02, 1.6785e-02, 9.8762e-01, 4.1225e-02, 1.4806e-03,
        9.8883e-01, 3.8394e-03, 5.6228e-05, 2.3920e-02, 9.7746e-01, 9.9119e-01,
        3.4683e-03, 9.6440e-01, 5.4233e-03, 6.9786e-01, 6.8937e-03, 9.0952e-03,
        9.9508e-01, 2.7360e-03, 3.3192e-02, 3.0701e-02, 2.7928e-03, 9.7720e-01,
        7.4061e-03, 9.8003e-01, 1.6891e-02, 9.8265e-01, 4.9796e-02, 3.0093e-02,
        1.4683e-03, 9.8141e-01, 1.8749e-03, 9.3589e-03, 3.0833e-03, 7.3436e-03,
        2.5329e-02, 9.8970e-01, 2.0301e-04, 1.2641e-04, 1.6729e-01, 1.7698e-02,
        2.1872e-04, 1.2164e-02, 4.1592e-02, 1.1803e-03, 3.3742e-03, 1.8250e-01,
        8.5229e-03, 9.8196e-01, 8.3979e-04, 8.5159e-01, 9.7723e-04, 9.2131e-01,
        2.6449e-04, 9.7212e-01, 1.2548e-03, 6.2677e-01, 1.5177e-02, 1.0060e-02,
        8.3739e-01, 1.1186e-03, 4.1676e-03, 8.8335e-03, 3.9794e-01, 9.6608e-01,
        3.1726e-01, 9.8333e-01, 1.4938e-04, 1.1333e-02, 1.4410e-02, 7.3301e-04,
        7.4415e-03, 9.9313e-01, 5.3082e-

In [24]:
torch.round(preds)

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 1., 0., 1.,
        0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0.,
        0., 1., 0., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 0., 0., 0., 1., 1.,
        0., 0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
        0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1.,
        1., 1., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 1., 1.,
        0., 0., 0., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 

In [25]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [26]:
torch.round(preds)[:12]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [27]:
EPOCHS = 125

In [28]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [29]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [30]:
torch.round(preds)[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [31]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [32]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [33]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [34]:
preds[0]

tensor(0.9987, device='cuda:0', grad_fn=<SelectBackward>)

In [35]:
preds

tensor([9.9865e-01, 9.9983e-01, 9.9943e-01, 9.9998e-01, 9.9956e-01, 9.9996e-01,
        9.9997e-01, 9.9996e-01, 9.9801e-01, 9.9811e-01, 9.9992e-01, 9.9996e-01,
        9.9761e-01, 9.9981e-01, 9.9986e-01, 9.9917e-01, 9.9991e-01, 9.9954e-01,
        9.9999e-01, 9.9978e-01, 9.9890e-01, 9.9892e-01, 9.9996e-01, 9.9996e-01,
        9.9994e-01, 9.9998e-01, 9.9944e-01, 9.9996e-01, 9.9993e-01, 9.9980e-01,
        9.7412e-01, 9.9996e-01, 9.9825e-01, 9.9988e-01, 1.5950e-03, 9.9876e-01,
        9.9807e-01, 9.9995e-01, 1.1105e-03, 9.9960e-01, 9.9991e-01, 9.9994e-01,
        6.1473e-01, 9.9280e-01, 9.9916e-01, 9.9973e-01, 9.9947e-01, 9.8160e-01,
        9.9982e-01, 9.9996e-01, 2.6293e-02, 9.9998e-01, 9.9892e-01, 9.9992e-01,
        4.3458e-03, 9.9998e-01, 9.9959e-01, 9.9605e-01, 9.9091e-01, 9.9997e-01,
        9.9996e-01, 8.0353e-01, 1.3147e-01, 9.9995e-01, 9.9993e-01, 9.9984e-01,
        9.9997e-01, 9.9998e-01, 9.9668e-01, 2.5133e-01, 9.9997e-01, 5.5184e-02,
        9.9949e-01, 1.0000e+00, 9.9735e-

In [36]:
EPOCHS = 112

In [37]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [38]:
preds

tensor([0.9880, 0.9966, 0.9928, 0.9993, 0.9945, 0.9985, 0.9992, 0.9986, 0.9604,
        0.9843, 0.9983, 0.9989, 0.9810, 0.9966, 0.9971, 0.9914, 0.9979, 0.9936,
        0.9994, 0.9958, 0.9898, 0.9890, 0.9988, 0.9989, 0.9984, 0.9992, 0.9932,
        0.9990, 0.9984, 0.9966, 0.9428, 0.9990, 0.9857, 0.9972, 0.0142, 0.9867,
        0.9843, 0.9986, 0.0058, 0.9897, 0.9982, 0.9985, 0.6632, 0.9665, 0.9915,
        0.9952, 0.9930, 0.9137, 0.9967, 0.9988, 0.0649, 0.9993, 0.9887, 0.9982,
        0.0165, 0.9994, 0.9919, 0.9751, 0.9603, 0.9991, 0.9989, 0.6089, 0.4025,
        0.9986, 0.9984, 0.9970, 0.9992, 0.9993, 0.9719, 0.4760, 0.9992, 0.4174,
        0.9928, 0.9998, 0.9792, 0.9997, 0.9997, 0.0263, 0.9977, 0.9995, 0.9938,
        0.9990, 0.9096, 0.9843, 0.9994, 0.9937, 0.9603, 0.0387, 0.9990, 0.9992,
        0.9969, 0.9883, 0.9962, 0.9999, 0.9930, 0.0889, 0.9987, 0.9982, 0.9687,
        0.9825, 0.9953, 0.9863, 0.9995, 0.9993, 0.9990, 0.9988, 0.9301, 0.8482,
        0.9995, 0.9993, 0.0379, 0.9988, 

In [39]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        0., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [40]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [41]:
EPOCHS = 75

In [42]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [43]:
preds

tensor([0.9825, 0.9912, 0.9912, 0.9979, 0.9915, 0.9913, 0.9979, 0.9937, 0.3669,
        0.9840, 0.9966, 0.9976, 0.9695, 0.9957, 0.9941, 0.9905, 0.9939, 0.9888,
        0.9984, 0.9893, 0.9850, 0.9910, 0.9939, 0.9971, 0.9931, 0.9983, 0.9901,
        0.9973, 0.9960, 0.9941, 0.9812, 0.9974, 0.9785, 0.9920, 0.5328, 0.9615,
        0.9872, 0.9965, 0.0808, 0.7584, 0.9969, 0.9958, 0.8141, 0.9791, 0.9892,
        0.9822, 0.9820, 0.9683, 0.9895, 0.9975, 0.2243, 0.9971, 0.9743, 0.9965,
        0.0773, 0.9981, 0.9458, 0.9810, 0.9773, 0.9975, 0.9970, 0.5299, 0.9247,
        0.9962, 0.9949, 0.9955, 0.9975, 0.9982, 0.8216, 0.9310, 0.9966, 0.8584,
        0.9860, 0.9989, 0.9800, 0.9986, 0.9987, 0.3417, 0.9958, 0.9984, 0.9912,
        0.9954, 0.2855, 0.9844, 0.9983, 0.9916, 0.9565, 0.8038, 0.9971, 0.9978,
        0.9944, 0.9860, 0.9932, 0.9994, 0.9923, 0.9029, 0.9959, 0.9962, 0.9699,
        0.9770, 0.9910, 0.9913, 0.9984, 0.9965, 0.9961, 0.9973, 0.6384, 0.9712,
        0.9986, 0.9977, 0.1095, 0.9966, 

In [44]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.,
        0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1.,
        1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [45]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [46]:
EPOCHS = 50

In [47]:
torch.round(preds)[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [48]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [49]:
preds

tensor([6.1262e-03, 2.4836e-03, 1.4914e-03, 6.1388e-01, 5.0371e-03, 3.3883e-04,
        5.5071e-01, 7.6243e-04, 1.3199e-05, 2.9932e-03, 1.7024e-01, 7.9710e-01,
        6.3215e-04, 7.2158e-02, 5.7243e-04, 2.2822e-02, 1.1188e-03, 1.3154e-03,
        9.2999e-01, 4.3217e-04, 5.2893e-03, 1.9918e-03, 5.9376e-04, 3.1647e-01,
        1.6191e-03, 5.3895e-02, 1.8900e-03, 3.5015e-01, 5.5861e-03, 3.0400e-03,
        1.3264e-04, 1.7268e-01, 2.8213e-04, 1.6328e-03, 6.1385e-04, 2.0776e-03,
        2.2488e-03, 9.3741e-01, 4.5436e-05, 3.3545e-05, 3.9307e-03, 2.2588e-03,
        3.2783e-05, 1.2302e-03, 4.2970e-03, 2.5592e-04, 6.7042e-04, 1.4141e-02,
        1.6868e-03, 3.0864e-01, 2.1107e-04, 8.8985e-02, 1.7182e-04, 2.5117e-02,
        6.9271e-05, 1.1097e-01, 3.8477e-04, 2.6354e-02, 1.8431e-03, 9.0808e-04,
        2.1111e-02, 2.7465e-04, 6.7776e-04, 1.1270e-03, 2.4649e-02, 6.0301e-02,
        9.5197e-03, 1.3473e-01, 2.6110e-05, 1.7807e-03, 3.0251e-03, 1.1960e-04,
        1.1947e-03, 8.9504e-01, 7.1188e-

In [50]:
torch.round(preds)[:12]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [51]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [52]:
EPOCHS = 25

In [53]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [54]:
preds

tensor([0.9555, 0.9737, 0.9749, 0.9836, 0.9713, 0.9818, 0.9829, 0.9828, 0.9665,
        0.9606, 0.9789, 0.9811, 0.9593, 0.9763, 0.9822, 0.9634, 0.9807, 0.9719,
        0.9842, 0.9774, 0.9593, 0.9734, 0.9830, 0.9813, 0.9788, 0.9873, 0.9722,
        0.9815, 0.9818, 0.9777, 0.9770, 0.9829, 0.9705, 0.9771, 0.9029, 0.9436,
        0.9660, 0.9740, 0.9108, 0.9707, 0.9841, 0.9828, 0.9671, 0.9618, 0.9668,
        0.9743, 0.9686, 0.9359, 0.9733, 0.9821, 0.9057, 0.9828, 0.9707, 0.9804,
        0.8966, 0.9866, 0.9625, 0.9434, 0.9582, 0.9887, 0.9832, 0.9218, 0.9417,
        0.9854, 0.9767, 0.9766, 0.9858, 0.9866, 0.9694, 0.9302, 0.9852, 0.9542,
        0.9691, 0.9889, 0.9506, 0.9873, 0.9875, 0.9294, 0.9805, 0.9798, 0.9665,
        0.9828, 0.9468, 0.9506, 0.9885, 0.9747, 0.9550, 0.9313, 0.9797, 0.9855,
        0.9774, 0.9745, 0.9750, 0.9896, 0.9720, 0.9427, 0.9760, 0.9795, 0.9781,
        0.9572, 0.9671, 0.9775, 0.9829, 0.9858, 0.9853, 0.9845, 0.9507, 0.9509,
        0.9857, 0.9836, 0.8810, 0.9815, 

In [55]:
torch.round(preds)[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], device='cuda:0',
       grad_fn=<SliceBackward>)

In [56]:
y_train[:12]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1.], device='cuda:0')

In [57]:
torch.round(preds)

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1., 1., 

In [58]:
torch.round(preds)[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [59]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [60]:
EPOCHS = 125

In [61]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [62]:
preds

tensor([0.9998, 1.0000, 0.9999, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 1.0000,
        0.9998, 0.9999, 1.0000, 0.9999, 0.9999, 1.0000, 0.9998, 1.0000, 0.9999,
        1.0000, 1.0000, 0.9998, 0.9999, 1.0000, 1.0000, 1.0000, 1.0000, 0.9999,
        1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9973, 0.9999,
        0.9999, 0.9999, 0.9998, 1.0000, 1.0000, 1.0000, 1.0000, 0.9998, 0.9999,
        1.0000, 1.0000, 0.9987, 1.0000, 1.0000, 0.9997, 1.0000, 1.0000, 1.0000,
        0.9998, 1.0000, 1.0000, 0.9994, 0.9997, 1.0000, 1.0000, 0.9998, 0.9996,
        1.0000, 1.0000, 0.9999, 1.0000, 1.0000, 1.0000, 0.9991, 1.0000, 0.9999,
        0.9999, 1.0000, 0.9996, 1.0000, 1.0000, 0.9998, 1.0000, 1.0000, 0.9998,
        1.0000, 1.0000, 0.9994, 1.0000, 1.0000, 0.9999, 0.9994, 1.0000, 1.0000,
        0.9999, 1.0000, 0.9999, 1.0000, 0.9999, 0.9996, 1.0000, 1.0000, 1.0000,
        0.9999, 0.9999, 0.9999, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 0.9996,
        1.0000, 1.0000, 0.9997, 1.0000, 

In [63]:
torch.round(preds)[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [64]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [65]:
EPOCHS = 250

In [66]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [67]:
preds

tensor([1.9432e-03, 3.1786e-03, 1.3530e-04, 9.9975e-01, 2.5181e-03, 3.9536e-04,
        9.9952e-01, 1.9338e-03, 1.1669e-07, 2.7840e-04, 9.9551e-01, 9.9942e-01,
        2.9268e-05, 6.7805e-01, 9.7209e-05, 1.4985e-02, 1.1386e-03, 2.2319e-04,
        9.9992e-01, 6.7044e-05, 1.7427e-03, 1.0071e-04, 1.2547e-03, 9.9921e-01,
        6.7900e-03, 9.9297e-01, 2.9211e-04, 9.9922e-01, 6.2067e-02, 1.8709e-03,
        2.5354e-07, 9.9819e-01, 7.1736e-06, 1.8014e-03, 1.8338e-08, 6.9334e-04,
        1.2212e-04, 9.9986e-01, 4.9892e-09, 1.6372e-06, 4.7832e-03, 6.3833e-03,
        4.9175e-08, 1.8855e-05, 8.8320e-04, 3.6396e-05, 1.0088e-04, 3.0564e-04,
        1.6555e-03, 9.9855e-01, 5.7513e-07, 9.9925e-01, 5.1436e-06, 9.6271e-01,
        5.8096e-08, 9.9920e-01, 1.0398e-04, 5.2664e-03, 3.2428e-05, 1.5865e-03,
        9.9022e-01, 2.4749e-06, 2.2864e-06, 1.3910e-03, 9.9190e-01, 7.9831e-01,
        9.8311e-01, 9.9838e-01, 1.8088e-07, 9.2370e-06, 2.3456e-01, 1.7495e-07,
        2.2000e-04, 9.9998e-01, 9.7446e-

In [68]:
torch.round(preds)[:25]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0.], device='cuda:0', grad_fn=<SliceBackward>)

In [69]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [70]:
preds[0]

tensor(0.0019, device='cuda:0', grad_fn=<SelectBackward>)

In [71]:
preds

tensor([1.9432e-03, 3.1786e-03, 1.3530e-04, 9.9975e-01, 2.5181e-03, 3.9536e-04,
        9.9952e-01, 1.9338e-03, 1.1669e-07, 2.7840e-04, 9.9551e-01, 9.9942e-01,
        2.9268e-05, 6.7805e-01, 9.7209e-05, 1.4985e-02, 1.1386e-03, 2.2319e-04,
        9.9992e-01, 6.7044e-05, 1.7427e-03, 1.0071e-04, 1.2547e-03, 9.9921e-01,
        6.7900e-03, 9.9297e-01, 2.9211e-04, 9.9922e-01, 6.2067e-02, 1.8709e-03,
        2.5354e-07, 9.9819e-01, 7.1736e-06, 1.8014e-03, 1.8338e-08, 6.9334e-04,
        1.2212e-04, 9.9986e-01, 4.9892e-09, 1.6372e-06, 4.7832e-03, 6.3833e-03,
        4.9175e-08, 1.8855e-05, 8.8320e-04, 3.6396e-05, 1.0088e-04, 3.0564e-04,
        1.6555e-03, 9.9855e-01, 5.7513e-07, 9.9925e-01, 5.1436e-06, 9.6271e-01,
        5.8096e-08, 9.9920e-01, 1.0398e-04, 5.2664e-03, 3.2428e-05, 1.5865e-03,
        9.9022e-01, 2.4749e-06, 2.2864e-06, 1.3910e-03, 9.9190e-01, 7.9831e-01,
        9.8311e-01, 9.9838e-01, 1.8088e-07, 9.2370e-06, 2.3456e-01, 1.7495e-07,
        2.2000e-04, 9.9998e-01, 9.7446e-

In [72]:
EPOCHS = 150

In [73]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [74]:
preds

tensor([0.9979, 0.9994, 0.9993, 0.9996, 0.9990, 0.9998, 0.9995, 0.9997, 0.9998,
        0.9982, 0.9993, 0.9993, 0.9988, 0.9992, 0.9997, 0.9984, 0.9997, 0.9993,
        0.9995, 0.9996, 0.9985, 0.9991, 0.9998, 0.9996, 0.9997, 0.9997, 0.9992,
        0.9995, 0.9996, 0.9993, 0.9995, 0.9995, 0.9994, 0.9995, 0.9904, 0.9984,
        0.9986, 0.9990, 0.9978, 0.9998, 0.9997, 0.9996, 0.9995, 0.9984, 0.9988,
        0.9997, 0.9992, 0.9942, 0.9995, 0.9995, 0.9973, 0.9997, 0.9996, 0.9995,
        0.9978, 0.9997, 0.9995, 0.9967, 0.9977, 0.9998, 0.9996, 0.9976, 0.9966,
        0.9998, 0.9994, 0.9991, 0.9997, 0.9997, 0.9998, 0.9947, 0.9998, 0.9987,
        0.9992, 0.9998, 0.9966, 0.9998, 0.9997, 0.9978, 0.9995, 0.9994, 0.9984,
        0.9997, 0.9995, 0.9965, 0.9998, 0.9994, 0.9987, 0.9960, 0.9994, 0.9997,
        0.9992, 0.9994, 0.9993, 0.9998, 0.9989, 0.9974, 0.9994, 0.9995, 0.9997,
        0.9985, 0.9990, 0.9994, 0.9996, 0.9998, 0.9998, 0.9997, 0.9990, 0.9973,
        0.9997, 0.9996, 0.9967, 0.9996, 

In [75]:
torch.round(preds)[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [76]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [77]:
EPOCHS = 175

In [78]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [79]:
preds

tensor([0.9701, 0.9866, 0.9827, 0.9866, 0.9797, 0.9949, 0.9842, 0.9931, 0.9968,
        0.9703, 0.9807, 0.9789, 0.9826, 0.9754, 0.9906, 0.9692, 0.9909, 0.9846,
        0.9832, 0.9905, 0.9751, 0.9780, 0.9947, 0.9877, 0.9915, 0.9868, 0.9821,
        0.9846, 0.9883, 0.9833, 0.9865, 0.9849, 0.9875, 0.9890, 0.9272, 0.9797,
        0.9730, 0.9761, 0.9810, 0.9966, 0.9877, 0.9899, 0.9910, 0.9718, 0.9758,
        0.9922, 0.9861, 0.9281, 0.9894, 0.9850, 0.9773, 0.9913, 0.9907, 0.9849,
        0.9830, 0.9908, 0.9913, 0.9533, 0.9645, 0.9935, 0.9887, 0.9787, 0.9651,
        0.9927, 0.9862, 0.9746, 0.9910, 0.9876, 0.9946, 0.9492, 0.9934, 0.9819,
        0.9851, 0.9928, 0.9564, 0.9914, 0.9901, 0.9785, 0.9859, 0.9809, 0.9710,
        0.9920, 0.9930, 0.9491, 0.9926, 0.9840, 0.9811, 0.9619, 0.9839, 0.9896,
        0.9804, 0.9866, 0.9830, 0.9903, 0.9733, 0.9684, 0.9853, 0.9859, 0.9921,
        0.9773, 0.9811, 0.9806, 0.9859, 0.9936, 0.9933, 0.9900, 0.9884, 0.9569,
        0.9874, 0.9872, 0.9769, 0.9878, 

In [80]:
torch.round(preds)[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [81]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [82]:
EPOCHS = 200

In [83]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [84]:
preds

tensor([8.6096e-01, 9.4588e-01, 3.7707e-01, 9.9984e-01, 9.6730e-01, 8.8204e-02,
        9.9985e-01, 7.9206e-01, 1.0288e-03, 1.9136e-01, 9.9951e-01, 9.9988e-01,
        1.5505e-02, 9.9871e-01, 2.6235e-01, 9.8804e-01, 9.0562e-01, 2.1391e-01,
        9.9996e-01, 3.3362e-02, 8.5188e-01, 4.0361e-01, 7.1257e-01, 9.9947e-01,
        9.4415e-01, 9.9977e-01, 5.4612e-01, 9.9971e-01, 9.9492e-01, 9.7962e-01,
        2.5990e-03, 9.9970e-01, 8.8704e-03, 9.1260e-01, 2.7757e-03, 6.1764e-02,
        1.4666e-01, 9.9985e-01, 5.4161e-04, 2.6809e-03, 9.9521e-01, 9.8774e-01,
        8.4674e-04, 1.9588e-02, 9.1553e-01, 1.4928e-02, 3.0315e-02, 2.6292e-01,
        7.6093e-01, 9.9967e-01, 2.9235e-03, 9.9908e-01, 6.6505e-03, 9.9864e-01,
        1.1405e-03, 9.9964e-01, 1.7490e-02, 9.4151e-01, 2.4489e-02, 9.8809e-01,
        9.9875e-01, 4.0450e-03, 5.8805e-03, 9.7528e-01, 9.9713e-01, 9.9885e-01,
        9.9851e-01, 9.9982e-01, 1.2070e-03, 1.2558e-02, 9.9271e-01, 1.7436e-03,
        9.0043e-02, 9.9995e-01, 7.3010e-

In [85]:
torch.round(preds)[:25]

tensor([1., 1., 0., 1., 1., 0., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1., 0.,
        1., 0., 1., 0., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [86]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [87]:
EPOCHS = 212

In [88]:
EPOCHS = 225

In [89]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [90]:
preds

tensor([1.1574e-02, 1.5840e-02, 1.4357e-03, 9.9937e-01, 2.2084e-02, 9.3582e-04,
        9.9910e-01, 6.0161e-03, 1.8254e-07, 1.8516e-03, 9.9628e-01, 9.9873e-01,
        9.5093e-05, 9.2240e-01, 8.8634e-04, 5.6176e-02, 5.7454e-03, 1.3388e-03,
        9.9957e-01, 3.4043e-04, 8.3675e-03, 1.0628e-03, 3.3518e-03, 9.9894e-01,
        2.3130e-02, 9.9713e-01, 2.8278e-03, 9.9880e-01, 9.3456e-01, 2.1263e-02,
        2.6522e-06, 9.9839e-01, 2.8945e-05, 9.4640e-03, 1.9147e-07, 1.4181e-03,
        9.5800e-04, 9.9921e-01, 1.5576e-08, 2.5476e-06, 1.0616e-01, 1.1575e-01,
        1.5676e-07, 1.6557e-04, 6.5362e-03, 1.0699e-04, 4.1154e-04, 2.3257e-03,
        5.8663e-03, 9.9842e-01, 1.0393e-06, 9.9896e-01, 1.6954e-05, 9.8645e-01,
        9.4641e-08, 9.9917e-01, 1.8236e-04, 2.1385e-02, 2.7335e-04, 2.1157e-02,
        9.9588e-01, 4.1174e-06, 1.0978e-05, 1.1496e-02, 9.9451e-01, 9.7722e-01,
        9.9464e-01, 9.9858e-01, 4.4925e-07, 5.5784e-05, 9.0420e-01, 7.8458e-07,
        1.0459e-03, 9.9990e-01, 5.9906e-

In [91]:
torch.round(preds)[:25]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0.], device='cuda:0', grad_fn=<SliceBackward>)

In [92]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [93]:
EPOCHS = 237

In [94]:
EPOCHS = 237

In [95]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [96]:
preds

tensor([5.5819e-04, 2.8326e-04, 6.2539e-05, 9.8949e-01, 4.3737e-04, 2.8420e-05,
        9.7961e-01, 1.4418e-04, 4.7761e-08, 1.6036e-04, 3.1676e-01, 9.8086e-01,
        1.2128e-05, 8.3767e-03, 1.9991e-05, 2.5457e-03, 9.6015e-05, 5.8040e-05,
        9.9570e-01, 1.3960e-05, 3.7197e-04, 7.3834e-05, 5.8934e-05, 9.6447e-01,
        3.1743e-04, 3.7959e-02, 9.6696e-05, 9.6502e-01, 1.0298e-03, 2.9507e-04,
        4.0898e-07, 8.8783e-01, 3.3252e-06, 1.6466e-04, 3.9444e-07, 1.2917e-04,
        9.3437e-05, 9.9460e-01, 1.8802e-08, 3.1438e-07, 3.5796e-04, 3.5255e-04,
        4.9175e-08, 2.1855e-05, 3.0600e-04, 5.9947e-06, 2.5248e-05, 6.3497e-04,
        1.2389e-04, 9.3540e-01, 6.0892e-07, 9.4782e-01, 1.7383e-06, 4.5896e-03,
        8.3873e-08, 9.4530e-01, 1.5831e-05, 2.6439e-03, 4.1464e-05, 9.6853e-05,
        1.1988e-02, 1.6419e-06, 4.7123e-06, 1.0156e-04, 2.8734e-02, 9.9041e-03,
        4.1818e-03, 8.1438e-01, 6.6344e-08, 2.0346e-05, 7.2938e-04, 2.5736e-07,
        5.5266e-05, 9.9882e-01, 6.5599e-

In [97]:
torch.round(preds)[:25]

tensor([0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
        1., 0., 0., 0., 0., 1., 0.], device='cuda:0', grad_fn=<SliceBackward>)

In [98]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [99]:
EPOCHS = 212

In [100]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [101]:
preds

tensor([9.8601e-01, 9.9877e-01, 9.3843e-01, 9.9999e-01, 9.9681e-01, 9.9916e-01,
        9.9998e-01, 9.9972e-01, 9.8272e-05, 5.9647e-01, 9.9990e-01, 9.9998e-01,
        7.9422e-03, 9.9932e-01, 9.9524e-01, 9.9524e-01, 9.9924e-01, 9.8631e-01,
        1.0000e+00, 9.8496e-01, 9.8665e-01, 7.5634e-02, 9.9964e-01, 9.9998e-01,
        9.9973e-01, 9.9997e-01, 9.8398e-01, 9.9998e-01, 9.9986e-01, 9.9856e-01,
        1.4424e-04, 9.9997e-01, 3.2496e-03, 9.9890e-01, 6.1286e-06, 9.5147e-01,
        5.1428e-02, 9.9998e-01, 2.1197e-06, 2.8309e-03, 9.9941e-01, 9.9977e-01,
        3.4037e-05, 2.6914e-03, 9.8659e-01, 9.1255e-01, 9.6271e-01, 1.0141e-02,
        9.9777e-01, 9.9997e-01, 6.8597e-05, 9.9999e-01, 2.9409e-03, 9.9985e-01,
        1.0449e-05, 9.9999e-01, 8.1083e-01, 9.4298e-01, 4.0970e-03, 9.9983e-01,
        9.9996e-01, 2.7183e-04, 3.4019e-04, 9.9964e-01, 9.9995e-01, 9.9945e-01,
        9.9997e-01, 9.9998e-01, 1.1800e-04, 7.7805e-04, 9.9995e-01, 6.0815e-05,
        9.8051e-01, 1.0000e+00, 9.3195e-

In [102]:
torch.round(preds)[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 1., 1., 1.,
        1., 1., 1., 0., 1., 1., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [103]:
y_train[:25]

tensor([1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 0.,
        1., 1., 0., 0., 0., 0., 1.], device='cuda:0')

In [104]:
torch.round(preds)[25:]

tensor([1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 1., 1., 0.,
        0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0., 1., 1.,
        0., 0., 1., 1., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 1., 0., 1., 1., 1., 0., 1.,
        1., 0., 0., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
        0., 1., 0., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 1.,
        1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 1., 1., 1., 1.,
        0., 0., 1., 0., 1., 1., 1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1.,
        1., 1., 0., 1.], device='cuda:0', grad_fn=<SliceBackward>)

In [105]:
y_train[25:]

tensor([0., 0., 1., 0., 1., 0., 1., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 0.,
        0., 1., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 0., 1., 1.,
        0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1.,
        1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1.,
        0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 0., 0.,
        1., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 0., 1.,
        1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 1., 1., 1., 0., 1., 0., 0., 0.,
        0., 1., 0., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1., 1., 1., 0., 1., 1.,
        1., 1., 1., 0., 0., 0., 0., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0.,
        0., 0., 0., 1., 0., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 1., 1.,
        0., 0., 0., 1., 1., 0., 1., 0., 0., 1., 0., 1., 0., 0., 0., 1., 0., 1.,
        1., 1., 1., 1.], device='cuda:0')

In [106]:
preds = torch.round(preds)

In [107]:
correct = 0
total = 

In [108]:
correct = 0
total = 0

In [109]:
for real,pred in zip(y_train,pred):
    if real == pred:
        correct += 1
    total += 1

In [110]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [111]:
correct

151

In [112]:
total

227

In [113]:
round(correct/total,3)

0.665

In [114]:
# EPOCHS = 212
EPOCHS = 200

In [115]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [116]:
preds

tensor([8.5414e-02, 2.2439e-01, 5.4154e-03, 9.9971e-01, 6.6586e-01, 5.5013e-03,
        9.9969e-01, 3.9204e-02, 5.1079e-06, 7.2458e-03, 9.9889e-01, 9.9977e-01,
        7.8575e-04, 9.9496e-01, 5.9430e-03, 9.1961e-01, 4.3867e-02, 8.3665e-03,
        9.9993e-01, 2.0274e-03, 6.5621e-02, 3.1125e-03, 2.0636e-02, 9.9929e-01,
        3.0551e-01, 9.9920e-01, 1.2963e-02, 9.9943e-01, 9.8736e-01, 7.4508e-01,
        2.7835e-05, 9.9928e-01, 2.9285e-04, 6.3507e-02, 2.6410e-06, 7.1964e-03,
        3.3252e-03, 9.9975e-01, 7.2833e-07, 3.3235e-05, 9.8174e-01, 9.4492e-01,
        4.8861e-06, 6.8531e-04, 3.9031e-02, 7.4531e-04, 2.3299e-03, 4.1765e-03,
        3.9237e-02, 9.9934e-01, 2.3043e-05, 9.9908e-01, 1.8071e-04, 9.9642e-01,
        3.6148e-06, 9.9948e-01, 1.1599e-03, 6.9331e-02, 1.1116e-03, 8.7083e-01,
        9.9796e-01, 6.6525e-05, 1.0542e-04, 4.3254e-01, 9.9652e-01, 9.9597e-01,
        9.9732e-01, 9.9949e-01, 9.9687e-06, 2.9444e-04, 9.7886e-01, 1.3411e-05,
        5.8321e-03, 9.9992e-01, 2.0085e-

In [117]:
preds = torch.round(preds)

In [118]:
correct = 0
total = 0

In [119]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [120]:
round(correct/total,3)

0.665

In [121]:
# EPOCHS = 212
EPOCHS = 225

In [122]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [123]:
preds

tensor([9.9988e-01, 9.9999e-01, 9.9991e-01, 1.0000e+00, 9.9997e-01, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 9.9667e-01, 9.9953e-01, 1.0000e+00, 1.0000e+00,
        9.9833e-01, 9.9999e-01, 9.9998e-01, 9.9996e-01, 1.0000e+00, 9.9994e-01,
        1.0000e+00, 9.9996e-01, 9.9992e-01, 9.9977e-01, 1.0000e+00, 1.0000e+00,
        1.0000e+00, 1.0000e+00, 9.9993e-01, 1.0000e+00, 1.0000e+00, 9.9999e-01,
        9.0429e-01, 1.0000e+00, 9.9857e-01, 9.9999e-01, 1.2511e-04, 9.9981e-01,
        9.9943e-01, 1.0000e+00, 3.7545e-04, 9.9981e-01, 1.0000e+00, 1.0000e+00,
        2.4265e-02, 9.9387e-01, 9.9992e-01, 9.9994e-01, 9.9988e-01, 9.8202e-01,
        9.9999e-01, 1.0000e+00, 9.1291e-03, 1.0000e+00, 9.9926e-01, 1.0000e+00,
        3.1428e-03, 1.0000e+00, 9.9992e-01, 9.9965e-01, 9.9149e-01, 1.0000e+00,
        1.0000e+00, 1.1435e-01, 1.6548e-02, 1.0000e+00, 1.0000e+00, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 9.9257e-01, 2.2645e-02, 1.0000e+00, 8.1267e-03,
        9.9992e-01, 1.0000e+00, 9.9925e-

In [124]:
preds = torch.round(preds)

In [125]:
correct = 0
total = 0

In [126]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [127]:
round(correct/total,3)

0.599

In [128]:
# EPOCHS = 212
EPOCHS = 250

In [129]:
# EPOCHS = 212
EPOCHS = 175

In [130]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [131]:
preds

tensor([9.3465e-01, 9.9408e-01, 5.0596e-02, 9.9994e-01, 9.8455e-01, 9.9455e-01,
        9.9992e-01, 9.9807e-01, 8.0618e-05, 3.1674e-02, 9.9961e-01, 9.9990e-01,
        4.2709e-03, 9.9765e-01, 7.8822e-01, 9.8740e-01, 9.9473e-01, 6.3402e-01,
        9.9997e-01, 1.3312e-01, 9.4379e-01, 1.2489e-02, 9.9795e-01, 9.9986e-01,
        9.9847e-01, 9.9983e-01, 6.2389e-01, 9.9988e-01, 9.9907e-01, 9.9052e-01,
        6.8588e-05, 9.9986e-01, 1.6205e-03, 9.9379e-01, 1.2316e-05, 5.2984e-01,
        1.3127e-02, 9.9990e-01, 2.9867e-06, 8.7569e-04, 9.9743e-01, 9.9834e-01,
        1.8392e-05, 1.9785e-03, 8.3560e-01, 1.7853e-02, 3.5942e-02, 1.6370e-02,
        9.9142e-01, 9.9983e-01, 7.9860e-05, 9.9992e-01, 1.3072e-03, 9.9937e-01,
        1.3366e-05, 9.9994e-01, 8.8728e-02, 7.9917e-01, 1.9962e-03, 9.9857e-01,
        9.9976e-01, 2.4911e-04, 2.8998e-04, 9.9753e-01, 9.9964e-01, 9.9807e-01,
        9.9980e-01, 9.9991e-01, 7.1318e-05, 7.5277e-04, 9.9961e-01, 3.7602e-05,
        5.5349e-01, 9.9999e-01, 1.1025e-

In [132]:
preds = torch.round(preds)

In [133]:
correct = 0
total = 0

In [134]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [135]:
round(correct/total,3)

0.683

In [136]:
# EPOCHS = 212
EPOCHS = 150

In [137]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()

In [138]:
preds

tensor([9.9668e-01, 9.9938e-01, 9.9739e-01, 9.9998e-01, 9.9903e-01, 9.9960e-01,
        9.9996e-01, 9.9973e-01, 2.6602e-02, 9.9098e-01, 9.9990e-01, 9.9994e-01,
        9.5160e-01, 9.9970e-01, 9.9918e-01, 9.9880e-01, 9.9962e-01, 9.9762e-01,
        9.9998e-01, 9.9803e-01, 9.9720e-01, 9.9430e-01, 9.9978e-01, 9.9996e-01,
        9.9974e-01, 9.9996e-01, 9.9774e-01, 9.9996e-01, 9.9987e-01, 9.9946e-01,
        6.9243e-03, 9.9995e-01, 9.3821e-01, 9.9947e-01, 4.2673e-04, 9.9286e-01,
        9.8938e-01, 9.9994e-01, 1.1568e-04, 9.6151e-01, 9.9983e-01, 9.9982e-01,
        1.5249e-03, 6.8518e-01, 9.9795e-01, 9.9668e-01, 9.9547e-01, 8.2238e-01,
        9.9930e-01, 9.9995e-01, 1.6853e-03, 9.9998e-01, 9.5828e-01, 9.9989e-01,
        4.2189e-04, 9.9998e-01, 9.9415e-01, 9.9243e-01, 8.0771e-01, 9.9990e-01,
        9.9995e-01, 6.7366e-03, 4.8897e-03, 9.9981e-01, 9.9991e-01, 9.9971e-01,
        9.9996e-01, 9.9997e-01, 1.2036e-02, 1.0629e-02, 9.9993e-01, 1.8467e-03,
        9.9703e-01, 1.0000e+00, 9.9106e-

In [139]:
preds = torch.round(preds)

In [140]:
correct = 0
total = 0

In [141]:
for real,pred in zip(y_train,preds):
    if real == pred:
        correct += 1
    total += 1

In [142]:
round(correct/total,3)

0.626

In [143]:
# EPOCHS = 212
EPOCHS = 125

In [144]:
model = Test_Model().to(device)
optimizer = torch.optim.SGD(model.parameters(),lr=0.25)
criterion = nn.L1Loss()
wandb.init(project=PROJECT_NAME,name='baseline')
for _ in tqdm(range(EPOCHS)):
    preds = model(X_train.float().to(device))
    preds = preds.view(len(preds),)
    preds.to(device)
    loss = criterion(preds,y_train)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    wandb.log({'loss':loss.item(),'val_loss':get_loss(criterion,X_test,y_test,model),'accuracy':get_accuracy(X_train,y_train,model),'val_accuracy':get_accuracy(X_test,y_test,model)})
wandb.finish()